In [47]:
import sys
import csv
import os
import shutil
import pandas as pd
import numpy as np
import decimal
from pyneuroml import pynml
from pyneuroml.pynml import print_comment_v
from pyneuroml.lems import LEMSSimulation
import neuroml as nml
import neuroml.writers as writers
from neuroml.utils import validate_neuroml2
import random
random.seed(12345)



os.getcwd()

'C:\\Users\\roche\\Thesis code nml'

## Rather complex but official way to generate a network file from a nml morphology

It is faster to take an existing net file and then with a short python script change the Cell_id everywhere to create the other networks,
this allows you to easily loop over cells and networks and separate them into different folders.

Normally, this method generates a file that refers to `<include href="C51A_scaled_exp_resample_5.cell.nml"/>`, meaning cells and networks should share the same folder. This creates a big mess when you want many networks. 

Therefore the file generated here will refer to `<include href="../Cells/C51A_scaled_exp_resample_5.cell.nml"/>` , thus the cell file should **not** be in the same folder as the network file, rather it should be one directory up and placed in a cells folder.


In [49]:
cell_id = 'C51A'
Temp = 37
net_id = "network_of_%s"%cell_id    
net_doc = nml.NeuroMLDocument(id='net_'+str(cell_id))

#include cell file from ../Cells/
cellfile = nml.IncludeType(href='../Cells/'+cell_id+'_scaled_exp_resample_5.cell.nml')
net_doc.includes.append(cellfile)

#create network
net = nml.Network(id=net_id, type="networkWithTemperature", temperature=str(Temp)+"degC")
net_doc.networks.append(net)

#create population
pop = nml.Population(id='population_of_%s'%cell_id,component=cell_id,type='populationList',size='1')
net.populations.append(pop)

loc = nml.Location(x='0',y='0',z='0')
inst = nml.Instance(id='0',location=loc)
pop.instances.append(inst)

#include current clamp 
t_delay = 50
t_duration = 200
Amp = "0.15nA"

Iclamp = nml.PulseGenerator(id='iclamp0',delay=str(t_delay)+'ms',duration=str(t_duration), amplitude=Amp)
net_doc.pulse_generators.append(Iclamp)

#add Iclamp to cell
input_list = nml.InputList(id='Iclamp', component=Iclamp.id, population=pop.id)
input = nml.Input(id='0', target = "../%s/0/"%(pop.id)+str(cell_id), segmentId="0", destination="synapses")
input_list.input.append(input)
net.input_lists.append(input_list)


#write the network file
net_file_name = 'nml_'+cell_id+'.net.nml'
writers.NeuroMLWriter.write(net_doc,net_file_name)
shutil.move('C:\\Users\\roche\\Thesis code nml\\'+str(net_file_name), 'C:\\Users\\roche\\Thesis code nml\\pynmlNetworks')

'C:\\Users\\roche\\Thesis code nml\\pynmlNetworks\\nml_C51A.net.nml'

population_of_C51A
